In [4]:
from pprint import pprint
import json
import sys
sys.path.insert(0, '../genepattern-python/')
from gp import functions

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%%HTML
<!--!AUTO_EXEC-->
<paper-material elevation="1" class="task-widget"><div class="task-widget-header item-header"><h2>Login GenePattern (Broad Institute)</h2></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="task-widget-inner" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="task-widget-content item-content"><div class="widget-info">Login into the GenePattern server at the Broad Institure.</div><form is="iron-form" class="task-widget-form"><div class="widget-form-panel"><div class="field-group field-required_args-group"><div class="item-header"><h3>Input</h3></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="field-group-content item-content"><div class="input-parent"><paper-input tabindex="0" aria-disabled="false" label="User name" name="required_args" value="Simpli" auto-validate="" error-message="Required!" required="required"></paper-input><paper-icon-button role="button" tabindex="0" aria-disabled="false" icon="info" class="info-toggle"></paper-icon-button></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="item-content">GenePattern user name.</div></iron-collapse><div class="input-parent"><paper-input tabindex="0" aria-disabled="false" label="Passcode" name="required_args" value="Simpli" auto-validate="" error-message="Required!" required="required"></paper-input><paper-icon-button role="button" tabindex="0" aria-disabled="false" icon="info" class="info-toggle"></paper-icon-button></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="item-content">GenePattern passcode.</div></iron-collapse></div></iron-collapse></div></div><button class="form-submit-button-wrapper"><paper-button role="button" tabindex="0" animated="" aria-disabled="false" elevation="1" class="form-submit-button" raised="">submit<iron-icon icon="assessment"></iron-icon></paper-button></button></form></div></iron-collapse></paper-material>
<!--{"Login GenePattern (Broad Institute)":{"library_path":"/Users/kateme/Jumis/tools/genepattern-python","library_name":"gp.functions","function_name":"login","description":"Login into the GenePattern server at the Broad Institure.","required_args":[{"name":"user_name","value":"Simpli","label":"User name","description":"GenePattern user name."},{"name":"passcode","value":"Simpli","label":"Passcode","description":"GenePattern passcode."}],"default_args":[{"name":"server_name","value":"https://genepattern.broadinstitute.org/gp","label":"Server Name","description":"No description."}],"optional_args":[],"returns":[]}}-->

Logged into GenePattern server at: https://genepattern.broadinstitute.org/gp.


In [2]:
%%HTML
<!--!AUTO_EXEC-->
<paper-material elevation="1" class="task-widget"><div class="task-widget-header item-header"><h2>PreprocessDataset</h2></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="task-widget-inner" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="task-widget-content item-content"><div class="widget-info">Performs several preprocessing steps on a res, gct, or odf input file</div><form is="iron-form" class="task-widget-form"><div class="widget-form-panel"><div class="field-group field-required_args-group"><div class="item-header"><h3>Input</h3></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="field-group-content item-content"><div class="input-parent"><paper-input tabindex="0" aria-disabled="false" label="Input.Filename" name="required_args" value="/Users/kateme/Jumis/data/animals/animals.gct" auto-validate="" error-message="Required!" required="required"></paper-input><paper-icon-button role="button" tabindex="0" aria-disabled="false" icon="info" class="info-toggle"></paper-icon-button></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="item-content">Input filename - .res, .gct, .odf</div></iron-collapse><div class="input-parent"><paper-input tabindex="0" aria-disabled="false" label="Threshold.For.Removing.Rows" name="required_args" value="''" auto-validate="" error-message="Required!" required="required"></paper-input><paper-icon-button role="button" tabindex="0" aria-disabled="false" icon="info" class="info-toggle"></paper-icon-button></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="item-content">Threshold for removing rows</div></iron-collapse><div class="input-parent"><paper-input tabindex="0" aria-disabled="false" label="Number.Of.Columns.Above.Threshold" name="required_args" value="''" auto-validate="" error-message="Required!" required="required"></paper-input><paper-icon-button role="button" tabindex="0" aria-disabled="false" icon="info" class="info-toggle"></paper-icon-button></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="item-content">Remove row if this number of columns not &gt;= given threshold</div></iron-collapse></div></iron-collapse></div><div class="field-group field-optional_args-group"><div class="item-header"><h3>Optional Input</h3></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="field-group-content item-content"><div class="input-parent"><paper-input tabindex="0" aria-disabled="false" label="Threshold.And.Filter" name="optional_args" value="1"></paper-input><paper-icon-button role="button" tabindex="0" aria-disabled="false" icon="info" class="info-toggle"></paper-icon-button></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="item-content">Apply floor, ceiling and variation filter</div></iron-collapse><div class="input-parent"><paper-input tabindex="0" aria-disabled="false" label="Floor" name="optional_args" value="20"></paper-input><paper-icon-button role="button" tabindex="0" aria-disabled="false" icon="info" class="info-toggle"></paper-icon-button></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="item-content">Value for floor threshold</div></iron-collapse><div class="input-parent"><paper-input tabindex="0" aria-disabled="false" label="Ceiling" name="optional_args" value="20000"></paper-input><paper-icon-button role="button" tabindex="0" aria-disabled="false" icon="info" class="info-toggle"></paper-icon-button></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="item-content">Value for ceiling threshold</div></iron-collapse><div class="input-parent"><paper-input tabindex="0" aria-disabled="false" label="Min.Fold.Change" name="optional_args" value="3"></paper-input><paper-icon-button role="button" tabindex="0" aria-disabled="false" icon="info" class="info-toggle"></paper-icon-button></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="item-content">Minimum fold change for variation filter</div></iron-collapse><div class="input-parent"><paper-input tabindex="0" aria-disabled="false" label="Min.Delta" name="optional_args" value="100"></paper-input><paper-icon-button role="button" tabindex="0" aria-disabled="false" icon="info" class="info-toggle"></paper-icon-button></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="item-content">Minimum delta for variation filter</div></iron-collapse><div class="input-parent"><paper-input tabindex="0" aria-disabled="false" label="Num.Outliers.To.Exclude" name="optional_args" value="0"></paper-input><paper-icon-button role="button" tabindex="0" aria-disabled="false" icon="info" class="info-toggle"></paper-icon-button></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="item-content">Number of outliers per row to ignore when calculating row min and max for variation filter</div></iron-collapse><div class="input-parent"><paper-input tabindex="0" aria-disabled="false" label="Row.Normalization" name="optional_args" value="0"></paper-input><paper-icon-button role="button" tabindex="0" aria-disabled="false" icon="info" class="info-toggle"></paper-icon-button></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="item-content">Perform row normalization</div></iron-collapse><div class="input-parent"><paper-input tabindex="0" aria-disabled="false" label="Row.Sampling.Rate" name="optional_args" value="1"></paper-input><paper-icon-button role="button" tabindex="0" aria-disabled="false" icon="info" class="info-toggle"></paper-icon-button></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="item-content">Sample rows without replacement to obtain this fraction of the total number of rows</div></iron-collapse><div class="input-parent"><paper-input tabindex="0" aria-disabled="false" label="Log2.Transform" name="optional_args" value="0"></paper-input><paper-icon-button role="button" tabindex="0" aria-disabled="false" icon="info" class="info-toggle"></paper-icon-button></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="item-content">Apply log2 transform after all other preprocessing steps</div></iron-collapse><div class="input-parent"><paper-input tabindex="0" aria-disabled="false" label="Output.File.Format" name="optional_args" value="3"></paper-input><paper-icon-button role="button" tabindex="0" aria-disabled="false" icon="info" class="info-toggle"></paper-icon-button></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="item-content">Output file format</div></iron-collapse><div class="input-parent"><paper-input tabindex="0" aria-disabled="false" label="Output.File" name="optional_args" value="<input.filename_basename>.preprocessed"></paper-input><paper-icon-button role="button" tabindex="0" aria-disabled="false" icon="info" class="info-toggle"></paper-icon-button></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="item-content">Output file name</div></iron-collapse></div></iron-collapse></div><div class="field-group field-returns-group"><div class="item-header"><h3>Output</h3></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="field-group-content item-content"><div class="input-parent"><paper-input tabindex="0" aria-disabled="false" label="Job" name="returns" value="j" auto-validate="" error-message="Required!" required="required"></paper-input><paper-icon-button role="button" tabindex="0" aria-disabled="false" icon="info" class="info-toggle"></paper-icon-button></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="item-content">ONLY FOR TESTING: RETURNS FIELD WILL BE REMOVED</div></iron-collapse></div></iron-collapse></div></div><button class="form-submit-button-wrapper"><paper-button role="button" tabindex="0" animated="" aria-disabled="false" elevation="1" class="form-submit-button" raised="">submit<iron-icon icon="assessment"></iron-icon></paper-button></button></form></div></iron-collapse></paper-material>
<!--{"PreprocessDataset":{"library_path":"/Users/kateme/Jumis/tools/genepattern-python","library_name":"gp.functions","function_name":"run_job","description":"Performs several preprocessing steps on a res, gct, or odf input file","required_args":[{"name":"input.filename","value":"/Users/kateme/Jumis/data/animals/animals.gct","label":"Input.Filename","description":"Input filename - .res, .gct, .odf"},{"name":"threshold.for.removing.rows","value":"''","label":"Threshold.For.Removing.Rows","description":"Threshold for removing rows"},{"name":"number.of.columns.above.threshold","value":"''","label":"Number.Of.Columns.Above.Threshold","description":"Remove row if this number of columns not >= given threshold"}],"default_args":[{"name":"task_name","value":"PreprocessDataset","label":"Task Name","description":"No description."}],"optional_args":[{"name":"threshold.and.filter","value":"1","label":"Threshold.And.Filter","description":"Apply floor, ceiling and variation filter"},{"name":"floor","value":"20","label":"Floor","description":"Value for floor threshold"},{"name":"ceiling","value":"20000","label":"Ceiling","description":"Value for ceiling threshold"},{"name":"min.fold.change","value":"3","label":"Min.Fold.Change","description":"Minimum fold change for variation filter"},{"name":"min.delta","value":"100","label":"Min.Delta","description":"Minimum delta for variation filter"},{"name":"num.outliers.to.exclude","value":"0","label":"Num.Outliers.To.Exclude","description":"Number of outliers per row to ignore when calculating row min and max for variation filter"},{"name":"row.normalization","value":"0","label":"Row.Normalization","description":"Perform row normalization"},{"name":"row.sampling.rate","value":"1","label":"Row.Sampling.Rate","description":"Sample rows without replacement to obtain this fraction of the total number of rows"},{"name":"log2.transform","value":"0","label":"Log2.Transform","description":"Apply log2 transform after all other preprocessing steps"},{"name":"output.file.format","value":"3","label":"Output.File.Format","description":"Output file format"},{"name":"output.file","value":"<input.filename_basename>.preprocessed","label":"Output.File","description":"Output file name"}],"returns":[{"label":"Job","description":"ONLY FOR TESTING: RETURNS FIELD WILL BE REMOVED","value":"j"}]}}-->

b'{"lsid": "urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00020:5.1", "params": [{"name": "input.filename", "values": ["https://genepattern.broadinstitute.org/gp/users/Simpli/tmp/run3131939259916255494.tmp/animals.gct"]}, {"name": "threshold.for.removing.rows", "values": []}, {"name": "number.of.columns.above.threshold", "values": []}, {"name": "threshold.and.filter", "values": [1]}, {"name": "floor", "values": [20]}, {"name": "ceiling", "values": [20000]}, {"name": "min.fold.change", "values": [3]}, {"name": "min.delta", "values": [100]}, {"name": "num.outliers.to.exclude", "values": [0]}, {"name": "row.normalization", "values": [0]}, {"name": "row.sampling.rate", "values": [1]}, {"name": "log2.transform", "values": [0]}, {"name": "output.file.format", "values": [3]}, {"name": "output.file", "values": ["<input.filename_basename>.preprocessed"]}], "tags": ["GenePattern Python Client"]}'


In [7]:
%%HTML
<!--!AUTO_EXEC-->
<paper-material elevation="1" class="task-widget"><div class="task-widget-header item-header"><h2>Report Job Statuses</h2></div><iron-collapse role="group" aria-hidden="true" aria-expanded="false" class="task-widget-inner" style="transition-property: max-height; transition-duration: 0s; max-height: 0px;"><div class="task-widget-content item-content"><div class="widget-info">Report job statuses.</div><form is="iron-form" class="task-widget-form"><div class="widget-form-panel"></div><button class="form-submit-button-wrapper"><paper-button role="button" tabindex="0" animated="" aria-disabled="false" elevation="1" class="form-submit-button" raised="">submit<iron-icon icon="assessment"></iron-icon></paper-button></button></form></div></iron-collapse></paper-material>
<!--{"Report Job Statuses":{"library_path":"/Users/kateme/Jumis/tools/genepattern-python","library_name":"gp.functions","function_name":"report_job_statuses","description":"Report job statuses.","required_args":[],"default_args":[],"optional_args":[],"returns":[]}}-->

Task: PreprocessDataset
	Status: The job is queued or being scheduled and executed


In [ ]:
functions.login('https://genepattern.broadinstitute.org/gp', 'Simpli', 'Simpli')

tasks = []
task_json = {
    "library_path": "Jumis/tools/genepattern-python",
    "tasks": tasks,
}

tasks.append(
    {
        "function_path": 'gp.functions.login',
        "label": 'Login GenePattern (Broad Institute)',
        "description": 'Login into the GenePattern server at the Broad Institure.',
        "required_args":
        [            
            {
                'name': 'user_name',
                'label': 'User name',
                "description": 'GenePattern user name.',
            },
            {
                'name': 'passcode',
                'label': 'Passcode',
                "description": 'GenePattern passcode.',
            },        
        ],
        "default_args":
        [
            {
                'name': 'server_name',
                'value': 'https://genepattern.broadinstitute.org/gp',
            },
        ],
    }
            )

for i, t in enumerate(functions.get_tasks()):
    print('Processing task ({}/280): {} ...'.format(i, t))
    
    t = functions.GPTask(functions.SERVER, t)
    t.param_load()
    name = t.get_name()

    required_args = []
    default_args = [{'name': 'task_name', 'value': name}]
    optional_args = []
    other_information = {}

    task_entry = {
        "function_path": 'gp.functions.execute',
        "label": name,
        "description": t.get_description(),
        "required_args": required_args,
        "default_args": default_args,
        "optional_args": optional_args,
        "returns": [{'label': 'Job', 'description': 'ONLY FOR TESTING: RETURNS FIELD WILL BE REMOVED'}],
        "other_information": other_information,
    }

    ps = t.get_parameters()
    for p in ps:
        a = {}
        description = p.get_description()
        default_value = p.get_default_value()

        a['name'] = p.get_name()
        a['description'] = description
        if default_value:
            a['value'] = default_value

        type_ = p.get_type()
        if p.is_choice_param():
            choices = [d['value'] for d in p.get_choices()]
            a['description']: '{} (default={}) (choices={}) (type={})'.format(description, default_value, choices, type_,)
        else:
            a['description']: '{} (default={}) (type={})'.format(description, default_value, type_,)

        if default_value:
            optional_args.append(a)
        else:
            required_args.append(a)

    other_information['lsid'] = t.get_lsid()
    other_information['version'] = t.get_version()

    tasks.append(task_entry)

In [ ]:
pprint(tasks)
with open('/Users/kateme/Jumis/tools/genepattern-python/functions.json', 'w') as f:
    json.dump(task_json, f, indent=4)

In [ ]:
js = functions.make_task_spec('PreprocessDataset')
# js.set_parameter('input.filename',
#                  'https://software.broadinstitute.org/cancer/software/genepattern/data/all_aml/all_aml_test.gct')
js.set_parameter('input.filename',
                 '/Users/kateme/Jumis/data/animals/animals.gct')
j = functions.submit_job(js)

In [ ]:
print(j.is_finished())
for f in j.get_output_files():
    print(f.read())

In [ ]:
import sys
sys.path.insert(0, '')
import simpli
m = simpli.Manager(True)
m.print_tasks_as_json()
simpli.default_tasks.link_json('/Users/kateme/Jumis/tools/genepattern-python/functions.json')